In [1]:
import pandas as pd
from datetime import datetime
from pandas.tseries.offsets import MonthEnd
import matplotlib.pyplot as plt

pd.set_option('display.float_format', '{:.2f}'.format)

In [25]:
df = pd.read_excel('../data/raw/consumo.xls')

# drop rows with NaN values
df = df.dropna()

# rename columns: "Região", "Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho", "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro", "Total"
df.columns = ["Região", "Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho", "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro", "Total"]

# reset index 
df = df.reset_index(drop=True)

# Rename "TOTAL BRASIL" from column "Região" to "Brasil"
df.loc[df['Região'] == 'TOTAL BRASIL', 'Região'] = 'Brasil'

df

,Região,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro,Total
0,Brasil,46835420.17,46309139.48,47827070.54,47337699.26,0,0,0,0,0,0,0,0,188309329.45
1,Norte,3469530.56,3313458.33,3466495.47,3455924.38,0,0,0,0,0,0,0,0,13705408.74
2,Nordeste,8401502.12,8159710.76,8208486.81,8354166.96,0,0,0,0,0,0,0,0,33123866.65
3,Sudeste,22368249.72,22055645.83,23022452.70,22685707.97,0,0,0,0,0,0,0,0,90132056.22
4,Sul,8896396.66,9152087.22,9355275.22,9077472.57,0,0,0,0,0,0,0,0,36481231.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,Sistemas Isolados,537166.33,515912.15,525471.52,538284.47,536324.74,539349.62,552987.79,559476.95,583883.24,605454.49,598478.62,603585.35,6696375.27
227,Norte,1795206.56,1707570.83,1815858.38,1813827.84,1864450.35,1855964.81,1880414.32,1955054.82,1877031.93,1934410.53,1895128.83,1921259.37,22316178.57
228,Nordeste,3719426.79,3444707.48,3629039.56,3733031.14,3671927.06,3678192.44,3633963.60,3680491.54,3823779.58,3870290.34,3961920.55,4011821.36,44858591.44
229,Sudeste/C.Oeste,16416445.30,16149752.45,16415189.55,17087727.76,16448778.17,16452891.05,16679846.07,16808652.04,17293344.91,17555268.08,17382176.82,17360225.99,202050298.19


In [26]:
# create empty dataframe 
df_filter_total = pd.DataFrame()

# filter the dataframe in a loop, showing groups of 12 rows
for i in range(0, len(df), 11):
    df_filter = df.iloc[i:i+11].copy().reset_index(drop=True)
    # drop rows 5 to 10
    df_filter = df_filter.drop(df_filter.index[6:11])
     
    # concat to total
    df_filter_total = pd.concat([df_filter_total, df_filter])
        
df_filter_total = df_filter_total.reset_index(drop=True)
df_filter_total

,Região,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro,Total
0,Brasil,46835420.17,46309139.48,47827070.54,47337699.26,0,0,0,0,0,0,0,0,188309329.45
1,Norte,3469530.56,3313458.33,3466495.47,3455924.38,0,0,0,0,0,0,0,0,13705408.74
2,Nordeste,8401502.12,8159710.76,8208486.81,8354166.96,0,0,0,0,0,0,0,0,33123866.65
3,Sudeste,22368249.72,22055645.83,23022452.70,22685707.97,0,0,0,0,0,0,0,0,90132056.22
4,Sul,8896396.66,9152087.22,9355275.22,9077472.57,0,0,0,0,0,0,0,0,36481231.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,Norte,1574434.34,1507448.59,1574176.69,1598862.11,1625478.92,1641381.91,1665002.57,1729895.89,1687156.54,1724166.75,1719638.46,1740620.86,19788263.63
122,Nordeste,4457112.38,4139768.00,4372743.11,4461999.48,4423584.52,4409301.74,4376174.12,4438514.59,4570030.34,4657880.99,4708464.83,4770509.72,53786083.82
123,Sudeste,14929524.40,14714713.51,14903795.68,15501322.96,14906628.72,14898517.41,15094285.46,15189574.87,15593685.74,15839877.16,15804794.18,15750647.95,183127368.04
124,Sul,4510113.04,4690227.86,4898770.40,4873311.14,4723026.16,4556563.56,4598792.89,4653182.44,4654411.58,4590697.50,4529419.23,4665214.36,55943730.16


In [27]:
# Current year
current_year = datetime.now().year

# Number of rows per group
rows_per_group = 6  # Since there are 6 regions: Brasil, Norte, Nordeste, Sudeste, Sul, Centro-Oeste

# Calculate the number of groups
num_groups = len(df_filter_total) // rows_per_group

In [28]:
years = []

# Assign years to each row
for i in range(num_groups):
    year = current_year - i
    years.extend([year] * rows_per_group)

# If there are remaining rows that form an incomplete group, add the remaining year
remaining_rows = len(df_filter_total) % rows_per_group
if remaining_rows > 0:
    years.extend([current_year - num_groups] * remaining_rows)

In [29]:
df_filter_total['Ano'] = years
df_filter_total

,Região,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro,Total,Ano
0,Brasil,46835420.17,46309139.48,47827070.54,47337699.26,0,0,0,0,0,0,0,0,188309329.45,2024
1,Norte,3469530.56,3313458.33,3466495.47,3455924.38,0,0,0,0,0,0,0,0,13705408.74,2024
2,Nordeste,8401502.12,8159710.76,8208486.81,8354166.96,0,0,0,0,0,0,0,0,33123866.65,2024
3,Sudeste,22368249.72,22055645.83,23022452.70,22685707.97,0,0,0,0,0,0,0,0,90132056.22,2024
4,Sul,8896396.66,9152087.22,9355275.22,9077472.57,0,0,0,0,0,0,0,0,36481231.67,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,Norte,1574434.34,1507448.59,1574176.69,1598862.11,1625478.92,1641381.91,1665002.57,1729895.89,1687156.54,1724166.75,1719638.46,1740620.86,19788263.63,2004
122,Nordeste,4457112.38,4139768.00,4372743.11,4461999.48,4423584.52,4409301.74,4376174.12,4438514.59,4570030.34,4657880.99,4708464.83,4770509.72,53786083.82,2004
123,Sudeste,14929524.40,14714713.51,14903795.68,15501322.96,14906628.72,14898517.41,15094285.46,15189574.87,15593685.74,15839877.16,15804794.18,15750647.95,183127368.04,2004
124,Sul,4510113.04,4690227.86,4898770.40,4873311.14,4723026.16,4556563.56,4598792.89,4653182.44,4654411.58,4590697.50,4529419.23,4665214.36,55943730.16,2004


In [30]:
# List of month columns (assuming they are named exactly as 'January', 'February', ..., 'December')
month_columns = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

# Melt the dataframe to long format
df_long = pd.melt(df_filter_total, id_vars=['Região', 'Ano'], value_vars=month_columns, 
                  var_name='Mês', value_name='Energia')

# Display the transformed dataframe
df_long

,Região,Ano,Mês,Energia
0,Brasil,2024,Janeiro,46835420.17
1,Norte,2024,Janeiro,3469530.56
2,Nordeste,2024,Janeiro,8401502.12
3,Sudeste,2024,Janeiro,22368249.72
4,Sul,2024,Janeiro,8896396.66
...,...,...,...,...
1507,Norte,2004,Dezembro,1740620.86
1508,Nordeste,2004,Dezembro,4770509.72
1509,Sudeste,2004,Dezembro,15750647.95
1510,Sul,2004,Dezembro,4665214.36


In [31]:
# Map month names to month numbers
month_map = {
    'Janeiro': 1,
    'Fevereiro': 2,
    'Março': 3,
    'Abril': 4,
    'Maio': 5,
    'Junho': 6,
    'Julho': 7,
    'Agosto': 8,
    'Setembro': 9,
    'Outubro': 10,
    'Novembro': 11,
    'Dezembro': 12
}

# Apply the month_map to create a month number column
df_long['Mês'] = df_long['Mês'].map(month_map)

# Pegar a estação do ano e criar uma coluna Estação 
def get_season(month):
    if month in [12, 1, 2]:
        return 'Verão'
    elif month in [3, 4, 5]:
        return 'Outono'
    elif month in [6, 7, 8]:
        return 'Inverno'
    else:
        return 'Primavera'
    
df_long['Estação'] = df_long['Mês'].apply(get_season)
df_long

,Região,Ano,Mês,Energia,Estação
0,Brasil,2024,1,46835420.17,Verão
1,Norte,2024,1,3469530.56,Verão
2,Nordeste,2024,1,8401502.12,Verão
3,Sudeste,2024,1,22368249.72,Verão
4,Sul,2024,1,8896396.66,Verão
...,...,...,...,...,...
1507,Norte,2004,12,1740620.86,Verão
1508,Nordeste,2004,12,4770509.72,Verão
1509,Sudeste,2004,12,15750647.95,Verão
1510,Sul,2004,12,4665214.36,Verão


In [32]:
# drop rows with 0 values
df_final = df_long[df_long['Energia'] != 0]
df_final.reset_index(drop=True)

df_final = df_final[['Ano', 'Mês', 'Região', 'Estação', 'Energia']]

df_final = df_final.sort_values(by=['Região', 'Ano', 'Mês']).reset_index(drop=True)

df_final

,Ano,Mês,Região,Estação,Energia
0,2004,1,Brasil,Verão,26978358.02
1,2004,2,Brasil,Verão,26508170.77
2,2004,3,Brasil,Outono,27284329.41
3,2004,4,Brasil,Outono,28046182.35
4,2004,5,Brasil,Outono,27244506.48
...,...,...,...,...,...
1459,2023,12,Sul,Verão,8365523.17
1460,2024,1,Sul,Verão,8896396.66
1461,2024,2,Sul,Verão,9152087.22
1462,2024,3,Sul,Outono,9355275.22


In [33]:
# save csv
df_final.to_csv('../data/processed/consumo_energia.csv', index=False)

In [ ]:
df = pd.read_excel('../data/raw/consumo.xls')